In [64]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from transformers import BertModel, BertTokenizer
from sentence_transformers import SentenceTransformer

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score
from torch.optim.lr_scheduler import StepLR

nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)

True

In [62]:
!kaggle datasets download -d julian3833/jigsaw-toxic-comment-classification-challenge -q

Dataset URL: https://www.kaggle.com/datasets/julian3833/jigsaw-toxic-comment-classification-challenge
License(s): CC0-1.0


In [24]:
import zipfile
import os

dir_path = '/Users/77771/Desktop/Projects/ToxicityClassifier'
zip_file = f'{dir_path}/jigsaw-toxic-comment-classification-challenge.zip'

# Распаковка архива
with zipfile.ZipFile(zip_file, 'r') as zips:
    zips.extractall(dir_path)

# Вывод содержимого распакованной папки
os.listdir(dir_path)


['.git',
 '.ipynb_checkpoints',
 'Classifier.ipynb',
 'jigsaw-toxic-comment-classification-challenge.zip',
 'README.md',
 'sample_submission.csv',
 'test.csv',
 'test_labels.csv',
 'train.csv']

In [61]:
train_data = pd.read_csv(f'{dir_path}/train.csv').drop('id', axis=1)
train_data.shape

(159571, 7)

In [33]:
train_data.head(5)

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [44]:
labels_columns = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
# train_data = train_data.drop((train_data[labels_columns]).sum(axis=1)==0, axis=0)
# train_data = train_data.drop(0, axis=0)

In [47]:
train_data.head(5)

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [59]:
train_data = train_data[(train_data[labels_columns]).sum(axis=1)!=0]
train_data.shape

(16225, 7)